In [1]:
import os
import random
from time import time
from glob import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import dill as pickle

from plotly import graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from matplotlib import pyplot as plt
import cv2

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T, models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
#!pip install -q torchsummary --user
# from torchsummary import summary


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cpu


In [2]:
def get_data(df_train):
    
    encoder = MultiLabelBinarizer()
    ohe_tags_train = encoder.fit_transform(df_train.list_tags.values)
#    ohe_tags_val = encoder.transform(df_val.list_tags.values)

    transform_train = get_transforms()
    ds_train = AmazonDataset(df_train, ohe_tags_train, transform_train, path=path_train)
#     ds_val = AmazonDataset(df_val, ohe_tags_val, transform_val, path=path_train)

    dl_train = DataLoader(
      ds_train,
      batch_size=64,
      shuffle=True,
      collate_fn=ds_train.collate_fn
    )
#     dl_val = DataLoader(
#       ds_val,
#       batch_size=64,
#       shuffle=True,
#       collate_fn=ds_val.collate_fn
#     )

    return ds_train, dl_train, encoder

In [3]:
def get_transforms():
    transform_test = T.Compose([
      T.ToPILImage(),
      T.Resize(224),
      T.ToTensor(),
      T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
      )
    ])
    return transform_test

In [4]:
class AmazonDatasetError(Exception):
    pass


class AmazonDataset(Dataset):
    def __init__(self, df, ohe_tags, transform, path, is_train=True, idx_tta=None):
        super().__init__()
        self.df = df
        self.ohe_tags = ohe_tags
        self.transform = transform
        if isinstance(path, str):
            self.paths = [path]
        elif isinstance(path, (list, tuple)):
            self.paths = path
        else:
            raise AmazonDatasetError(f"Path type must be str, list or tuple, got: {type(path)}")
        self.is_train = is_train
        if not is_train:
            if not idx_tta in list(range(6)):
                raise AmazonDatasetError(
                    f"In test mode, 'idx_tta' must be an int belonging to [0, 5], got: {repr(idx_tta)}"
                )
            self.idx_tta = idx_tta

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filename = self.df.iloc[idx].image_name + ".jpg"
        for path in self.paths:
            if filename in os.listdir(path):
                file_path = os.path.join(path, filename)
                break
        else:
            raise AmazonDatasetError(f"Can't fetch {filename} among {self.paths}")
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.ohe_tags[idx]
        return img, label

    def collate_fn(self, batch):
        imgs, labels = [], []
        for (img, label) in batch:
            img = self.custom_augment(img)
            img = torch.tensor(img)
            img = img.permute(2, 0, 1)
            img = self.transform(img)
            imgs.append(img[None])
            labels.append(label)
        imgs = torch.cat(imgs).float().to(device)
        labels = torch.tensor(labels).float().to(device)
        return imgs, labels

    def load_img(self, idx, ax=None):
        img, ohe_label = self[idx]
        label = self.df.iloc[idx].tags
        title = f"{label} - {ohe_label}"
        if ax is None:
            plt.imshow(img)
            plt.title(title)
        else:
            ax.imshow(img)
            ax.set_title(title)
    
    def custom_augment(self, img):
        """
        Discrete rotation and horizontal flip.
        Random during training and non random during testing for TTA.
        Not implemented in torchvision.transforms, hence this function.
        """
        choice = np.random.randint(0, 6) if self.is_train else self.idx_tta
        if choice == 0:
            # Rotate 90
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_CLOCKWISE)
        if choice == 1:
            # Rotate 90 and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_CLOCKWISE)
            img = cv2.flip(img, flipCode=1)
        if choice == 2:
            # Rotate 180
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_180)
        if choice == 3:
            # Rotate 180 and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_180)
            img = cv2.flip(img, flipCode=1)
        if choice == 4:
            # Rotate 90 counter-clockwise
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
        if choice == 5:
            # Rotate 90 counter-clockwise and flip horizontally
            img = cv2.rotate(img, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
            img = cv2.flip(img, flipCode=1)
        return img

In [ ]:
def find_best_thresholds(Y_hat, Y):
    N_tags = Y.shape[1]
    best_threshs = [0.2] * N_tags
    resolution = 100
    for jdx in tqdm(range(N_tags)):
        best_score = 0
        #threshs = np.zeros_like(best_threshs)
        threshs = best_threshs.copy()
        for kdx in range(resolution):
            kdx /= resolution
            threshs[jdx] = kdx
            Y_hat_thresh = (Y_hat > threshs).astype(float)
            score = fbeta_score(Y, Y_hat_thresh, beta=2, average="samples")
            if score > best_score:
                best_score = score
                best_threshs[jdx] = kdx
    
    global_best_score = fbeta_score(Y, (Y_hat > best_threshs).astype(float), beta=2, average="samples")
    print(f"threshs: {best_threshs} -- best score: {global_best_score}")
    
    return best_threshs

In [7]:
df_test = pd.read_csv('/data2/ntua/data/planet/planet/test_2000.csv')

In [8]:
path = "/data2/ntua/data/planet/planet"
path_train = os.path.join(path, "train-jpg")

In [9]:
df_test["list_tags"] = df_test.tags.str.split(" ")

In [10]:
ds_test, dl_test, encoder = get_data(df_test)

In [11]:
ds_test.__len__()

2005

In [12]:
imgs, labels = next(iter(dl_test))
print('Images shape: {}, labels shape: {}'.format(imgs.shape, labels.shape))

Images shape: torch.Size([64, 3, 224, 224]), labels shape: torch.Size([64, 17])


In [13]:
model = torch.load("resnet50_fold0.pth")

In [14]:
@torch.no_grad()
def batch_predict(model, X):
    model.eval()
    Y = model(X)
    return Y.detach().float().cpu().numpy()

In [15]:
Y_hat_test = []
for idx_tta in range(1):#6
    Y_hat_test_tta = []
#    dl_test, df_test = get_test_data(idx_tta)
    for X, _ in tqdm(dl_test):
        Y_hat_test_batch = batch_predict(model, X)
        Y_hat_test_tta.extend(Y_hat_test_batch)
    Y_hat_test.append(Y_hat_test_tta)


  0%|          | 0/32 [00:00<?, ?it/s]

In [28]:
Y_val = np.array(train_results["Y_val"])

NameError: name 'train_results' is not defined

In [16]:
def global_best_score(Y_hat, Y, thresholds):
    return fbeta_score(Y, (Y_hat > thresholds).astype(float), beta=2, average="samples")    

In [17]:
threshs=[0.11,0.09,0.15,0.05,0.04,0.25,0.2,0.04,0.13,0.12,0.07,0.18,0.32,0.15,0.06,0.12,0.16]

In [21]:
dl_test

In [19]:
global_best_score(Y_hat_test, Y, thresholds)

NameError: name 'Y' is not defined

In [63]:
Y_hat_test1 = np.mean(np.array(Y_hat_test), axis=0)
Y_hat_test1[0]

array([5.1055215e-03, 9.6639286e-04, 2.7975568e-03, 3.6337695e-03,
       7.1147265e-04, 9.9463499e-01, 9.6316880e-04, 8.8265887e-04,
       6.5411548e-03, 7.3585258e-04, 2.1630938e-03, 2.0741711e-03,
       9.9798989e-01, 6.3718222e-03, 1.9416617e-03, 1.7733703e-03,
       5.0974358e-03], dtype=float32)

In [64]:
Y_hat_test2 = (Y_hat_test1 > threshs).astype(float)

In [65]:
len(Y_hat_test2)

1005

In [66]:
Y_hat_test_inv = encoder.inverse_transform(Y_hat_test2)
test_tags = []
for row in Y_hat_test_inv:
    tags = " ".join(row)
    test_tags.append(tags)

df_test["pr_tags"] = test_tags
df_test.to_csv("resnet50_submission.csv", index=False)